## Implement RSA

There are two annoying things about implementing RSA. Both of them involve key generation; the actual encryption/decryption in RSA is trivial.

First, you need to generate random primes. You can't just agree on a prime ahead of time, like you do in DH. You can write this algorithm yourself, but I just cheat and use OpenSSL's BN library to do the work.

The second is that you need an "invmod" operation (the multiplicative inverse), which is not an operation that is wired into your language. The algorithm is just a couple lines, but I always lose an hour getting it to work.

I recommend you not bother with primegen, but do take the time to get your own EGCD and invmod algorithm working.

Now:

* Generate 2 random primes. We'll use small numbers to start, so you can just pick them out of a prime table. Call them "p" and "q".
* Let n be p * q. Your RSA math is modulo n.
* Let et be (p-1)*(q-1) (the "totient"). You need this value only for keygen.
* Let e be 3.
* Compute d = invmod(e, et). invmod(17, 3120) is 2753.
* Your public key is [e, n]. Your private key is [d, n].
* To encrypt: `c = m**e%n`. To decrypt: `m = c**d%n`
* Test this out with a number, like "42".
* Repeat with bignum primes (keep e=3).

Finally, to encrypt a string, do something cheesy, like convert the string to hex and put "0x" on the front of it to turn it into a number. The math cares not how stupidly you feed it strings.


## Generate random primes

In [33]:
from Crypto.Util.number import getPrime

In [34]:
NUM_BITS = 1024
p = getPrime(NUM_BITS)
q = getPrime(NUM_BITS)

In [35]:
n = p * q
n.bit_length()

2048

In [36]:
et = (p - 1) * (q - 1) # Euler's totient.

In [37]:
e = 3 # public exponent.

## DIY egcd/invmod

In [38]:
def egcd(a, b):
    assert a >= 0 and b >= 0 and a + b > 0

    if b == 0:
        d, x, y = a, 1, 0
    else:
        d, p, q = egcd(b, a % b)
        x = q
        y = p - (a // b) * q

    assert a % d == 0 and b % d == 0
    assert d == a*x + b*y
    return (d, x, y)

In [39]:
def invmod(a, b, n):
    d, s, t = egcd(a, n)
    assert n > 1 and d == 1
    return b * s % n

In [40]:
#d = pow(e, -1, et)
d = invmod(e, 1, et)

In [41]:
assert e * d % et == 1

## Test with 42

In [42]:
m = 42

In [43]:
c = pow(m, e, n)
c

74088

In [44]:
pow(c, d, n)

42

## Test with strings

In [45]:
m = b'attack at dawn'

In [46]:
c = pow(int.from_bytes(m, 'big'), e, n)
c

7722709502790459166639993213708833737796147876436427049309008419119949153843426449996193173565875000

In [47]:
p = pow(c, d, n)
p

1976620216402300889624482718775150

In [48]:
p.to_bytes(len(m), 'big')

b'attack at dawn'

Try again with help from PyCryptodome.

In [49]:
from Crypto.Util.number import bytes_to_long, long_to_bytes

In [50]:
c = pow(bytes_to_long(m), e, n)
c

7722709502790459166639993213708833737796147876436427049309008419119949153843426449996193173565875000

In [51]:
long_to_bytes(pow(c, d, n))

b'attack at dawn'